In [27]:
from typing import Any
import numpy as np
from process_bigraph import Process, Composite, ProcessTypes, pp


class Toy(Process):
    config_schema = {
        'k': 'float'
    }
    def __init__(self, config: dict[str, Any] = None, core: ProcessTypes = None) -> None:
        super().__init__(config, core)
        self.k = self.config['k']

    def initial_state(self):
        return {
            'x': self.k,
            'y': self.k**self.k
        }

    def inputs(self):
        return {
            'x': 'float'
        }

    def outputs(self):
        return {
            'x': 'float',
            'y': 'float'
        }

    def update(self, state, interval):
        x = state['x'] * self.k
        return {
            'x': x,
            'y': x**x
        }

In [28]:
from bsp import app_registrar

app_registrar.core.register_process('toy', Toy)

In [29]:
config = {
    'k': 0.35
}

state = {
    'toy': {
        '_type': 'process',
        'address': 'local:toy',
        'config': config,
        'inputs': {
            'x': ['x_store'],
        },
        'outputs': {
            'x': ['x_store'],
            'y': ['y_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'x': 'float',
                'y': 'float',
            }
        },
        'inputs': {
            'x': ['x_store'],
            'y': ['y_store'],
        }
    }
}

In [30]:
sim = Composite(config={'state': state}, core=app_registrar.core)

In [31]:
sim.run(10)

In [32]:
results = sim.gather_results()

In [33]:
pp(results)

{ ('emitter',): [ {'x': 0.35, 'y': 0.6925064384421088},
                  {'x': 0.4725, 'y': 1.4657150296277588},
                  {'x': 0.637875, 'y': 2.208314187919128},
                  {'x': 0.8611312499999999, 'y': 2.9238252284962183},
                  {'x': 1.1625271874999998, 'y': 3.62047440265183},
                  {'x': 1.5694117031249997, 'y': 4.31405967575313},
                  {'x': 2.1187057992187492, 'y': 5.033635203474405},
                  {'x': 2.8602528289453115, 'y': 5.834762885376529},
                  {'x': 3.8613413190761703, 'y': 6.835852560963448},
                  {'x': 5.21281078075283, 'y': 8.338235469258873},
                  {'x': 7.03729455401632, 'y': 11.333572543663768}]}
